In [1]:
import pandas as pd
import numpy as np
from neuralforecast import NeuralForecast
import logging
import os
from ticker_data import get_df_for_period, get_ticker

predict_ticker = 'KO'
tickers = ['KO', '^SPX']

period_train = {'start': '2019-06-01', 'end': '2025-01-01'}
period_test  = {'start': '2025-01-01', 'end': '2025-11-01'}

df_train = get_df_for_period(tickers, period_train)
df_test  = get_df_for_period(tickers, period_test)
df_full  = pd.concat([df_train, df_test]).sort_values(['unique_id', 'ds']).reset_index(drop=True)

print(f"Training ends:    {df_train['ds'].max().date()}")
print(f"Test starts:      {df_test['ds'].min().date()}")

# Actual prices for P&L calculation (need one extra day beyond test end for next-day entry)
ts = pd.to_datetime(period_test['end'])          # Timestamp('2025-10-01 00:00:00')
later_ts = (ts + pd.Timedelta(days=2)).strftime('%Y-%m-%d') # Timestamp('2025-10-03 00:00:00')
    
traded_ticker_raw   = get_ticker(predict_ticker, start=period_test['start'], end=later_ts)
traded_ticker_open  = traded_ticker_raw['Open']
traded_ticker_close = traded_ticker_raw['Close']

# Test dates: unique trading days in the test period for the traded ticker
test_dates = sorted(df_test[df_test['unique_id'] == f'{predict_ticker}_price']['ds'].unique())

[*********************100%***********************]  2 of 2 completed
/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pandas/core/internals/blocks.py:347: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
[*********************100%***********************]  2 of 2 completed

unique_id
SPX_vol    2
unique_id
KO_price     1405
KO_open      1405
KO_high      1405
KO_low       1405
KO_vol       1405
SPX_price    1405
SPX_open     1405
SPX_high     1405
SPX_low      1405
SPX_vol      1403
Name: count, dtype: int64
Total days: 1405  |  Total rows: 14048  |  Series: 10
y range: [-1.489380, 1.625940]  |  mean: 0.000286  |  std: 0.120124



[*********************100%***********************]  1 of 1 completed

unique_id
KO_price     208
KO_open      208
KO_high      208
KO_low       208
KO_vol       208
SPX_price    208
SPX_open     208
SPX_high     208
SPX_low      208
SPX_vol      208
Name: count, dtype: int64
Total days: 208  |  Total rows: 2080  |  Series: 10
y range: [-1.328556, 1.069421]  |  mean: 0.000922  |  std: 0.115114
Training ends:    2024-12-31
Test starts:      2025-01-03


In [ ]:
from lib.trade import Trade
from matplotlib.pylab import mean

from lib.tradeparams import TradeSimParams, Trader, dump_params_to_json
from pathlib import Path
import json

simfolder = Path("simfolder/tradesimlog/...")
params:TradeSimParams = TradeSimParams.load_from_folder(simfolder)
trader = Trader()
portfolio_value = Trader.START_VALUE
current_pos = 0
position_return = 0
entry_price = None

FEE = params.FEE

SIGNAL_TRIGGER_STOPLOSS = -2
MODEL_INPUT_LEN = 45

# Log-Header
with open(params.sim_log_file, 'w') as f_sim_log_file:
    f_sim_log_file.write("date,pred_momentum,signal,in_market,trade_return,portfolio_value,position_return\n")

# Load pre-trained model
nf = NeuralForecast.load(path=params.model_path)
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)
logging.getLogger("neuralforecast").setLevel(logging.ERROR)
logging.getLogger("lightning.pytorch").setLevel(logging.ERROR)

predict_ticker = params.traded_symbol
# Several approaches for usign the confidence interval, from simple to more sophisticated:                                                                             
                                                                                                                                     
#   1. Skip uncertain days (flat when interval is wide)                                                                                
#   interval_width = cv['PatchTST-hi-80'] - cv['PatchTST-lo-80']                                                                     
#   threshold = interval_width.median()  # or a fixed value                                                                            
                                                                                                                                     
#   cv['signal'] = 0  # default: stay flat
#   cv.loc[(cv['PatchTST-median'] > 0) & (interval_width < threshold), 'signal'] = 1
#   cv.loc[(cv['PatchTST-median'] < 0) & (interval_width < threshold), 'signal'] = -1

#   2. Only trade when the interval agrees on direction
#   # Long only if even the pessimistic bound is positive
#   cv['signal'] = 0
#   cv.loc[cv['PatchTST-lo-80'] > 0, 'signal'] = 1   # entire 80% interval is positive
#   cv.loc[cv['PatchTST-hi-80'] < 0, 'signal'] = -1  # entire 80% interval is negative
#   This is the most conservative — you only trade when the model is highly confident.

#   3. Scale position size by confidence
#   max_width = (cv['PatchTST-hi-80'] - cv['PatchTST-lo-80']).max()
#   confidence = 1 - (cv['PatchTST-hi-80'] - cv['PatchTST-lo-80']) / max_width  # 0=uncertain, 1=very confident

#   cv['signal'] = np.where(cv['PatchTST-median'] > 0, 1, -1) * confidence
#   # signal is now a float between -1 and 1 — use as position size fraction

#   4. Use the 95% interval for stop-loss sizing
#   cv['signal'] = np.where(cv['PatchTST-median'] > 0, 1, -1)
#   # Expected worst case loss if wrong:
#   cv['risk'] = cv['PatchTST-hi-95'] - cv['PatchTST-lo-95']
#   # Skip if expected risk exceeds your stop-loss threshold
#   cv.loc[cv['risk'] > 0.02, 'signal'] = 0
def get_signal(forecast):
    predictions = [row for _, row in forecast.query(
                f"unique_id == '{predict_ticker}_price'"
             ).iloc[0:3].iterrows()]
    
    #preds = forecast.query(f"unique_id == '{predict_ticker}_price'").iloc[0,1,2,3] #get predictions for the next days    
    if 'PatchTST' in predictions[0]: #MAE-based prediction
        pred_avg_over_next_2_days = (predictions[0]['PatchTST'] + predictions[1]['PatchTST'])/2
        # Signal logic
        signal = 0
        if pred_avg_over_next_2_days > params.THRESHOLD:  signal = 1
        elif pred_avg_over_next_2_days < -1*params.THRESHOLD: signal = -1
        return (signal, pred_avg_over_next_2_days)
    else : #mq-loss trained net
        return get_signal_from_confidence(predictions)
        

def get_signal_from_confidence(predictions):
    signal = 0
    #if p1['PatchTST-lo-80'] > 0: 
    #    signal = 1   # entire 80% interval is positive        
    #if p1['PatchTST-hi-80'] < 0:
    #    signal = -1  # entire 80% interval is negative
    trend_pred = mean([p['PatchTST-median'] for p in predictions])
    if trend_pred > params.THRESHOLD: 
        signal = 1       
    if trend_pred < -1*params.THRESHOLD:
        signal = -1  
    print ([p['PatchTST-median'] for p in predictions])
    # pred_momentum < -1*params.THRESHOLD: signal = -1
    return signal, trend_pred

for i, today in enumerate(test_dates):
    # --- Build input window: last MODEL_INPUT_LEN rows per unique_id up to today ---
    df_list = []
    for uid in df_full['unique_id'].unique():
        series = df_full[df_full['unique_id'] == uid].sort_values('ds')
        window = series[series['ds'] <= today].tail(MODEL_INPUT_LEN)
        if len(window) > 0:
            df_list.append(window)

    df_step = pd.concat(df_list).reset_index(drop=True)

    # Predict next h days
    forecast = nf.predict(df=df_step)
    
    (signal, prediction) = get_signal(forecast)
    # --- P&L with real prices ---
    trade_return = 0.0

    if current_pos != 0:
        close_today = traded_ticker_close.loc[today]
        position_return = current_pos * (close_today - entry_price) / entry_price
        if position_return < params.STOPLOSS_THRESHOLD:
            signal = SIGNAL_TRIGGER_STOPLOSS
    else:
        position_return = 0

    this_position_return = position_return
    day_current_signal   = signal
    day_current_pos      = current_pos
    portfolio_value *= (1 + this_position_return)

    if signal != 0: #keep position until signal inverts, tp or stoploss
        if signal != current_pos:
            if current_pos != 0 and entry_price is not None:
                position_return = 0
                signal = 0
                current_pos = 0
            portfolio_value *= (1 - FEE)
            if signal != 0 and i + 1 < len(test_dates):
                next_day = test_dates[i + 1]
                entry_price = traded_ticker_open.loc[next_day]
                current_pos = signal
            else:
                entry_price = None

    print(f"{today.date()} | Pred: {prediction:.5f} | Signal: {signal} | "
          f"Pos: {current_pos} | Return: {trade_return:+.4f} | Port: {portfolio_value:.2f}")
    with open(log_file, 'a') as f:
        f.write(f"{today.date()},{prediction},{day_current_signal},{day_current_pos},{trade_return},{portfolio_value},{this_position_return}\n")

# Close last open position
if current_pos != 0 and entry_price is not None:
    last_close = traded_ticker_close.loc[test_dates[-1]]
    final_return = current_pos * (last_close - entry_price) / entry_price
    portfolio_value *= (1 + final_return)
    portfolio_value *= (1 - FEE)
    print(f"\nFinal close | Return: {final_return:+.4f} | Port: {portfolio_value:.2f}€")

result = {
    'start_value': START_VALUE, 
    'end_value': portfolio_value , 
    'yield': f"{(portfolio_value/START_VALUE-1)*100:+.2f}%"
    }

Path(results_file).write_text(json.dumps(result, indent=4))


print(f"\n=== Endwert: {portfolio_value:.2f} (Start: 10000€, Rendite: {(portfolio_value/10000-1)*100:+.2f}%) ===")

Seed set to 1


✅ Saved 9 items to test_ohlc/KO/from_trained_test_ohlc.json
[-0.00021252932492643595, 0.0005069718463346362, -0.001051439787261188]
2025-01-03 | Pred: -0.00025 | Signal: -1 | Pos: -1 | Return: +0.0000 | Port: 9995.00
[-0.0033312542364001274, 0.0014871051535010338, 0.005292554385960102]
2025-01-06 | Pred: 0.00115 | Signal: 0 | Pos: 0 | Return: +0.0000 | Port: 10102.09


/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pytorch_lightning/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pytorch_lightning/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pytorch_lightning/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pytorch_lightning/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pytorch_lightning/utiliti

[0.003071203362196684, 0.0055290525779128075, -0.0006199295748956501]
2025-01-07 | Pred: 0.00266 | Signal: 1 | Pos: 1 | Return: +0.0000 | Port: 10097.03
[-0.00046230293810367584, -0.0009807925671339035, 0.003449701704084873]
2025-01-08 | Pred: 0.00067 | Signal: 1 | Pos: 1 | Return: +0.0000 | Port: 10216.23
[-0.0015605781227350235, 0.00015577208250761032, 0.007305069826543331]
2025-01-10 | Pred: 0.00197 | Signal: 1 | Pos: 1 | Return: +0.0000 | Port: 10229.63


/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pytorch_lightning/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pytorch_lightning/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pytorch_lightning/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.


...

/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pytorch_lightning/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pytorch_lightning/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pytorch_lightning/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pytorch_lightning/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.


[-0.0034459689632058144, -0.0024493392556905746, 0.001729307696223259]
2025-10-23 | Pred: -0.00139 | Signal: -1 | Pos: -1 | Return: +0.0000 | Port: 13988.06
[-0.0014504875289276242, -0.00013976963236927986, -0.00362231838516891]
2025-10-24 | Pred: -0.00174 | Signal: -1 | Pos: -1 | Return: +0.0000 | Port: 14158.53
[-0.00249598640948534, -0.0013207178562879562, -0.0010041198693215847]
2025-10-27 | Pred: -0.00161 | Signal: -1 | Pos: -1 | Return: +0.0000 | Port: 14260.85
[-0.0033639902248978615, -0.0006715450435876846, -0.006528597325086594]
2025-10-28 | Pred: -0.00352 | Signal: -1 | Pos: -1 | Return: +0.0000 | Port: 14343.71


/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pytorch_lightning/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pytorch_lightning/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pytorch_lightning/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pytorch_lightning/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.


[0.007705718278884888, -0.004887431859970093, 0.0156887024641037]
2025-10-29 | Pred: 0.00617 | Signal: 0 | Pos: 0 | Return: +0.0000 | Port: 14787.53
[-0.005334692075848579, 0.014254644513130188, -0.001766212284564972]
2025-10-30 | Pred: 0.00238 | Signal: 1 | Pos: 1 | Return: +0.0000 | Port: 14780.14
[0.0033137451391667128, -0.0023423945531249046, -0.0005010556196793914]
2025-10-31 | Pred: 0.00016 | Signal: 0 | Pos: 1 | Return: +0.0000 | Port: 14855.60

Final close | Return: +0.0051 | Port: 14923.99€

=== Endwert: 14923.99 (Start: 10000€, Rendite: +49.24%) ===


/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pytorch_lightning/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
